
# Семинар: ClearML `Dataset` — практическое руководство (SDK)

Этот ноутбук превращает страницу документации **ClearML Dataset SDK** в структурированный практический конспект с примерами кода, комментариями, объяснениями параметров и ожидаемыми результатами.


## Содержание
1. [Подготовка окружения](#prep)
2. [Концепции и жизненный цикл датасета](#concepts)
3. [Создание: `Dataset.create`](#create)
4. [Получение: `Dataset.get` / `Dataset.list_datasets`](#get)
5. [Обновление версии: `Dataset.update`](#update)
6. [Добавление/удаление файлов: `add_files`, `add_external_files`, `remove_files`](#addremove)
7. [Метаданные: `set_description`, `add_tags`, `remove_tags`](#meta)
8. [Загрузка и фиксация: `upload`, `finalize`](#upload)
9. [Локальная копия: `get_local_copy`, `get_mutable_local_copy`](#localcopy)
10. [Инспекция содержимого: `list_added_files`, `list_removed_files`, `list_files`](#inspect)



<a id="prep"></a>

## 1. Подготовка окружения

- Установите пакет: `pip install clearml`
- Привяжите SDK к серверу: `clearml-init` (введите **API credentials** и **server URL**)
- (Опционально) Подключите S3/MinIO в `clearml.conf` или через переменные окружения.

**Ожидаемый результат:** команды SDK смогут создавать/получать датасеты, загружать файлы, а локальный кэш будет автоматически управляться ClearML.


In [ ]:

# !pip install clearml  # раскомментируйте при необходимости

from clearml import Dataset, Task

print("Импорт выполнен. Готово к работе с ClearML Dataset SDK.")


<a id="concepts"></a>

## 2. Концепции и жизненный цикл датасета

- **Версия датасета** — неизменяемый слепок набора файлов + метаданные.
- **Происхождение(Lineage)** — связь версий: каждая новая версия ссылается на родительские.
- **Финализация** — момент «заморозки» версии (после `finalize()` версия становится read‑only).
- **Хранилище** — File Server ClearML или внешнее (S3/MinIO/…); управление кэшем — прозрачно.



<a id="create"></a>

## 3. Создание: `Dataset.create`

```python
Dataset.create(
    dataset_name: str,
    dataset_project: str = None,
    dataset_tags: list[str] = None,
    parent_datasets: list[str] = None,
    description: str = None,
    readonly: bool = False,
    use_current_task: bool = False,
    dataset_id: str = None,    # обычно не указывают при создании
)
```
**Назначение:** создать *новую* рабочую версию датасета (до финализации её можно наполнять файлами и метаданными).

**Ключевые параметры:**
- `dataset_name` — логичное имя набора (например, "Cars-Detections").
- `dataset_project` — пространство проектов в ClearML (для группировки).
- `dataset_tags` — пользовательские ярлыки (быстрый поиск/фильтрация).
- `parent_datasets` — список ID родительских версий (наследование содержимого).
- `description` — текстовое описание изменений.
- `use_current_task` — связать датасет с текущей Task (удобно для пайплайнов).


In [ ]:
ds = Dataset.create(
    dataset_name="demo_dataset",
    dataset_project="Seminar3",
    dataset_tags=["raw", "v1"],
    description="Initial raw drop"
)
print(f"Создан черновик датасета: id={ds.id}, name={ds.name}")



<a id="get"></a>

## 4. Получение: `Dataset.get` / `Dataset.list_datasets`

```python
Dataset.get(
    dataset_id: str = None,
    dataset_name: str = None,
    dataset_project: str = None,
    tags: list[str] = None,
    only_completed: bool = False,  # искать только финализированные версии
)
```

```python
Dataset.list_datasets(
    dataset_project: str = None,
    dataset_name: str = None,
    only_completed: bool = False,
    tags: list[str] = None,
    partial_match: bool = False,
)
```
**Назначение:** искать и получать существующие версии по `id`/имени/тегам/проекту.

**Ожидаемый результат:** найден объект датасета (последняя версия при совпадении имени) или список версий для инспекции.


In [ ]:

# Поиск последней версии по имени/проекту
found = Dataset.get(dataset_name="demo_dataset", dataset_project="Seminar3")
print(f"Найдена версия: id={found.id}, name={found.name}")

# Листинг всех версий (включая незавершённые)
versions = Dataset.list_datasets(dataset_name="demo_dataset", dataset_project="Seminar3", only_completed=False)
print("Всего найдено версий:", len(versions))
for v in versions:
    print("-", v)



<a id="update"></a>

## 5. Обновление версии: `Dataset.update`

```python
Dataset.update(dataset_id: str)
```
**Назначение:** создать **новую версию** на основе существующей (аналог «commit» для данных).

**Ожидаемый результат:** появляется новая рабочая версия, связанная с предыдущей в lineage. После наполнения — `finalize()`.


In [ ]:

base = Dataset.get(dataset_name="demo_dataset", dataset_project="Seminar3")
updated = Dataset.update(dataset_id=base.id)
print(f"Создана новая версия на основе {base.id}: new_id={updated.id}")



<a id="addremove"></a>

## 6. Добавление/удаление файлов

### `add_files`
```python
dataset.add_files(
    path: str | list[str],
    dataset_path: str | None = None,  # куда поместить внутри датасета
    recursive: bool = True,
    verbose: bool = False,
)
```
**Назначение:** добавить локальные файлы/папки в текущую рабочую версию.

### `add_external_files`
```python
dataset.add_external_files(
    source_url: str | list[str],      # ссылки: s3://, gs://, http(s)://, azure:// ...
    dataset_path: str | None = None,
    overwrite: bool = False,
    verbose: bool = False,
)
```
**Назначение:** добавить файлы по внешним URL без предварительного локального скачивания (перекладывает в сторидж датасета).

### `remove_files`
```python
dataset.remove_files(
    path: str | list[str],            # путь внутри датасета (а не локальный путь)
    recursive: bool = True,
    verbose: bool = False,
)
```
**Назначение:** удалить ранее добавленные файлы/папки из рабочей версии.

**Ожидаемый результат:** состояние рабочей версии обновлено; сами данные будут загружены/синхронизированы на этапе `upload()`/`finalize()`.


In [ ]:

# Примеры (папки и пути — плейсхолдеры)
updated.add_files(path="data/train/", dataset_path="images/train", verbose=True)
updated.add_external_files(source_url=["s3://bucket/imgs/a.jpg", "https://host/path/b.jpg"],
                           dataset_path="images/ext", verbose=True)
updated.remove_files(path="images/old/", recursive=True, verbose=True)
print("Изменения в рабочей версии подготовлены.")



<a id="meta"></a>

## 7. Метаданные: описание и теги

```python
dataset.set_description(text: str)
dataset.add_tags(tags: list[str])
dataset.remove_tags(tags: list[str])
```
**Назначение:** задокументировать изменения и облегчить поиск.

**Ожидаемый результат:** в UI у версии появится описание и теги; ими удобно фильтровать и отслеживать историю.


In [ ]:

updated.set_description("Cleaned files, added external images, reorganized folders")
updated.add_tags(["v2", "clean", "augmented"])
# updated.remove_tags(["deprecated"])  # пример
print("Метаданные обновлены.")



<a id="upload"></a>

## 8. Загрузка и фиксация: `upload`, `finalize`

```python
dataset.upload(
    storage_uri: str | None = None,   # если не задано, используется конфиг ClearML (files_server/S3/...)
    verbose: bool = False,
)
dataset.finalize(verbose: bool = False)
```
**Назначение:**
- `upload()` — физическая загрузка добавленных/изменённых файлов в хранилище ClearML.
- `finalize()` — «закрыть» версию; после этого изменения недопустимы (версия immutable).

**Ожидаемый результат:** новая версия появится в UI как **Completed**; станет доступна по ID/тегам/имени для скачивания и использования в экспериментах.


In [ ]:
updated.upload(verbose=True)   # можно указать storage_uri="s3://bucket/prefix"
updated.finalize()
print("Версия финализирована и готова к использованию.")



<a id="localcopy"></a>

## 9. Локальная копия: `get_local_copy`, `get_mutable_local_copy`

```python
Dataset.get(...).get_local_copy(
    extract_archive: bool = True,
    force: bool = False,
) -> str
```
**Назначение:** скачать **read‑only** копию версии в локальный кэш; вернуть путь к папке с данными.

```python
Dataset.get(...).get_mutable_local_copy(
    target_folder: str | Path = None,
    overwrite: bool = False,
) -> str
```
**Назначение:** получить **редактируемую** локальную копию (в указанную папку), удобно для внесения правок перед `update()`.

**Ожидаемый результат:** появится локальная директория с данными. Повторные вызовы — быстрые, за счёт кэша.


In [ ]:

# Read-only копия
ro_path = Dataset.get(dataset_name="demo_dataset", dataset_project="Seminar3").get_local_copy()
print("Read-only кэш:", ro_path)

# Редактируемая копия в указанную папку
rw_path = Dataset.get(dataset_name="demo_dataset", dataset_project="Seminar3").get_mutable_local_copy(
    target_folder="/tmp/demo_dataset_v2", overwrite=False
)
print("Mutable копия:", rw_path)



<a id="inspect"></a>

## 10. Инспекция содержимого

Доступные вспомогательные методы могут отличаться по версиям SDK. Чаще всего используются:

- `Dataset.list_datasets(...)` — увидеть версии/фильтровать по тегам/статусу.
- `Dataset.get(...).list_files()` — перечислить файлы версии (если поддерживается SDK).
- `Dataset.get(...).list_added_files()` / `list_removed_files()` — изменения относительно родителей (если поддерживается SDK).

**Ожидаемый результат:** текстовые списки/таблицы в выводе, удобные для аудита и автоматических проверок.


In [ ]:
current = Dataset.get(dataset_name="demo_dataset", dataset_project="Seminar3")
try:
    files = current.list_files()
    print("Всего файлов:", len(files))
    print("Примеры путей:")
    for p in files[:10]:
        print(" -", p)
except Exception as e:
    print("list_files() недоступен в данной версии SDK/сервере:", e)
